In [1]:
import re
import wikipedia as wiki
from unidecode import unidecode
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
max_letters = 12
language_tags = {

                'en':['actor', 'alcohol', 'cheque', 'cancer', 'chocolate', 'debate', 'hobby', 'melon', 'propaganda',
                      'religion', 'violin', 'england', 'MediaWiki'],

                'si':["ශ්‍රී දළදා මාළිගාව",  "දියවඩන නිලමේ", "මහනුවර", "මාතලේ පරිපාලන දිස්ත්‍රික්කය", "ලංකා දිවයින","සංස්කෘතිය",
                      "න්‍යෂ්ටික අවි","තිරසාර සංවර්ධනය","කාන්තාව හා සංවර්ධනය","ආර්ථික සංවර්ධනය","මුල් ළමාවියේ සංවර්ධනය",
                     "ඵලදායී පාසල","රසායනික කර්මාන්ත"]

                 }

In [3]:
def generate_dictionary(tag, max_word_length):
    wiki.set_lang(tag)
    for topic in language_tags[tag]:
        page = wiki.WikipediaPage(topic)
        content = page.content
        content = unidecode(content)
        lst = process(content, max_word_length)
    return lst

In [4]:
def process(page_content, max_word_length):
    words = re.sub(r'[^a-zA-Z ]', '', page_content)
    lower = words.lower()
    word_list = lower.split()
    short_words = []
    for word in word_list:
        if len(word) <= max_word_length:
            short_words.append(word)
    return short_words

In [5]:
def convert_dic_to_vector(dic, max_word_length):
    new_list = []
    for word in dic:
        vec = ''
        n = len(word)
        for i in range(n):
            current_letter = word[i]
            ind = ord(current_letter)-97
            placeholder = (str(0)*ind) + str(1) + str(0)*(25-ind)
            vec = vec + placeholder
        if n < max_word_length:
            excess = max_word_length-n
            vec = vec +str(0)*26*excess
        new_list.append(vec)
    return new_list

In [6]:
def create_output_vector(tag_index, number_of_languages):
    out = str(0)*tag_index + str(1) + str(0)*(number_of_languages-1-tag_index)
    return out

In [7]:
word_data = []
language_data = []
master_dic = []

count = 0

for tag in language_tags.keys():
    print('generating dictionary for ' + tag)
    dic = generate_dictionary(tag, max_letters)
    for word in dic:
        master_dic.append(word)
    vct = convert_dic_to_vector(dic, max_letters)
    for vector in vct:
        word_data.append(vector)
    output_vct = create_output_vector(count, len(language_tags))
    for i in range(len(vct)):
        language_data.append(output_vct)
    count += 1

generating dictionary for en
generating dictionary for si


In [8]:
arr = []
for i in range(len(word_data)):
    entry = []
    entry.append(master_dic[i])
    for digit in language_data[i]:
        entry.append(float(digit))
    for digit in word_data[i]:
        entry.append(float(digit))
    arr.append(entry)


arr = np.array(arr)
print(len(arr[1]))
np.save('arr.npy', arr)
df=pd.DataFrame(arr)
df.to_csv('data.csv')
print("Done")
print(type(arr))

315
Done
<class 'numpy.ndarray'>


In [9]:
data = arr

inputs = data[:, 1+len(language_tags):]
labels = data[:, 1:1+len(language_tags)]

x_train, x_test, y_train, y_test = train_test_split(inputs, labels, test_size=0.15)

print(x_test.shape)
print(y_test.shape)
print(x_train.shape)
print(y_train.shape)

(952, 312)
(952, 2)
(5393, 312)
(5393, 2)


In [10]:

network = Sequential()
network.add(Dense(200, input_dim=26*max_letters, activation='sigmoid'))
network.add(Dense(150, activation='sigmoid'))
network.add(Dense(100, activation='sigmoid'))
network.add(Dense(100, activation='sigmoid'))
network.add(Dense(len(language_tags), activation='softmax'))

network.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tboard = TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
callbacks_list = [checkpoint, tboard]

network.fit(x_train, y_train, epochs=100, batch_size=1000, validation_data=(x_test, y_test), callbacks=callbacks_list)


Instructions for updating:
Use tf.cast instead.
Train on 5393 samples, validate on 952 samples
Epoch 1/100
5393/5393 [==============================] - 2s 433us/step - loss: 0.1985 - acc: 0.6049 - val_loss: 0.0825 - val_acc: 0.9191

Epoch 00001: val_acc improved from -inf to 0.91912, saving model to weights.hdf5
Epoch 2/100
5393/5393 [==============================] - 2s 344us/step - loss: 0.0665 - acc: 0.9319 - val_loss: 0.0750 - val_acc: 0.9191

Epoch 00002: val_acc did not improve from 0.91912
Epoch 3/100
5393/5393 [==============================] - 2s 342us/step - loss: 0.0641 - acc: 0.9319 - val_loss: 0.0770 - val_acc: 0.9191

Epoch 00003: val_acc did not improve from 0.91912
Epoch 4/100
5393/5393 [==============================] - 2s 345us/step - loss: 0.0652 - acc: 0.9319 - val_loss: 0.0778 - val_acc: 0.9191

Epoch 00004: val_acc did not improve from 0.91912
Epoch 5/100
5393/5393 [==============================] - 2s 338us/step - loss: 0.0657 - acc: 0.9319 - val_loss: 0.0782 - v


Epoch 00042: val_acc did not improve from 0.91912
Epoch 43/100
5393/5393 [==============================] - 2s 360us/step - loss: 0.0633 - acc: 0.9319 - val_loss: 0.0744 - val_acc: 0.9191

Epoch 00043: val_acc did not improve from 0.91912
Epoch 44/100
5393/5393 [==============================] - 2s 357us/step - loss: 0.0633 - acc: 0.9319 - val_loss: 0.0744 - val_acc: 0.9191

Epoch 00044: val_acc did not improve from 0.91912
Epoch 45/100
5393/5393 [==============================] - 2s 403us/step - loss: 0.0633 - acc: 0.9319 - val_loss: 0.0744 - val_acc: 0.9191

Epoch 00045: val_acc did not improve from 0.91912
Epoch 46/100
5393/5393 [==============================] - 2s 361us/step - loss: 0.0633 - acc: 0.9319 - val_loss: 0.0744 - val_acc: 0.9191

Epoch 00046: val_acc did not improve from 0.91912
Epoch 47/100
5393/5393 [==============================] - 2s 349us/step - loss: 0.0633 - acc: 0.9319 - val_loss: 0.0744 - val_acc: 0.9191

Epoch 00047: val_acc did not improve from 0.91912
Epoc

Epoch 86/100
5393/5393 [==============================] - 2s 348us/step - loss: 0.0367 - acc: 0.9574 - val_loss: 0.0484 - val_acc: 0.9391

Epoch 00086: val_acc improved from 0.93803 to 0.93908, saving model to weights.hdf5
Epoch 87/100
5393/5393 [==============================] - 2s 352us/step - loss: 0.0334 - acc: 0.9624 - val_loss: 0.0471 - val_acc: 0.9433

Epoch 00087: val_acc improved from 0.93908 to 0.94328, saving model to weights.hdf5
Epoch 88/100
5393/5393 [==============================] - 2s 348us/step - loss: 0.0310 - acc: 0.9661 - val_loss: 0.0455 - val_acc: 0.9422

Epoch 00088: val_acc did not improve from 0.94328
Epoch 89/100
5393/5393 [==============================] - 2s 344us/step - loss: 0.0293 - acc: 0.9664 - val_loss: 0.0445 - val_acc: 0.9454

Epoch 00089: val_acc improved from 0.94328 to 0.94538, saving model to weights.hdf5
Epoch 90/100
5393/5393 [==============================] - 2s 335us/step - loss: 0.0276 - acc: 0.9681 - val_loss: 0.0437 - val_acc: 0.9464

Epo